In [ ]:
print("hi")

hi


In [ ]:
dbutils.fs.unmount('/mnt/imageforcaptioning')

/mnt/imageforcaptioning has been unmounted.


True

In [ ]:
source_directory = 'wasbs://images@imagecaptioningmorvin.blob.core.windows.net'
mount_point = '/mnt/imageforcaptioning'
configuration_key = 'fs.azure.account.key.imagecaptioningmorvin.blob.core.windows.net'
account_key = ''
dbutils.fs.mount(source=source_directory,
                   mount_point=mount_point,
                   extra_configs={configuration_key: account_key})

True

In [ ]:
dbutils.library.restartPython()

In [ ]:
%pip install azureml-core
%pip install tensorflow
%pip install keras

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for azureml-core from https://files.pythonhosted.org/packages/f0/5d/a734ff6bd2ed2e865396fc3dffc2cfbc691ea88504fb3a5953000b6b3c63/azureml_core-1.56.0-py3-none-any.whl.metadata
  Obtaining dependency information for backports.tempfile from https://files.pythonhosted.org/packages/b4/5c/077f910632476281428fe254807952eb47ca78e720d059a46178c541e669/backports.tempfile-1.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for knack<0.12.0 from https://files.pythonhosted.org/packages/53/6b/caf27d5a40618c7e945a1c68e1961c2d3637edfce9ebb0edc27c9ff53c1c/knack-0.11.0-py3-none-any.whl.metadata
  Obtaining dependency information for pkginfo from https://files.pythonhosted.org/packages/c0/38/d617739840a2f576e400f03fea0a75703f93cc274002635b4b998bbb9de4/pkginfo-1.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for ar

In [ ]:

from azureml.core import Workspace
workspace_name = "imagecaptioning-aml" # replace with your Workspace name
workspace_location = 'Central India'
resource_group = 'image-captioning'
subscription_id = ''

ws = Workspace.create(name = workspace_name,
                             location = workspace_location,
                             resource_group = resource_group,
                             subscription_id = subscription_id,
                             exist_ok=True)

Performing interactive authentication. Please follow the instructions on the terminal.


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code NXLS65DSU to authenticate.


Interactive authentication successfully completed.


In [ ]:
import os
import urllib.request
from azureml.core.model import Model

registered_model = Model.register(workspace = ws,
                        model_path = '/dbfs/mnt/imageforcaptioning/model.h5',
                        model_name = 'image-captioning-model',
                        description = 'Automatically generate a caption for a given image')
registered_tokenizer = Model.register(workspace = ws,
                        model_path = '/dbfs/mnt/imageforcaptioning/tokenizer.pkl',
                        model_name = 'image-captioning-tokenizer',
                        description = 'Pickled tokenizer for image caption model')

Registering model image-captioning-model
Registering model image-captioning-tokenizer


In [ ]:
models = Model.list(ws)
for m in models:
  print(m.name, m.version)
print(Model.get_model_path('image-captioning-tokenizer',1, _workspace=ws))
print(Model.get_model_path('image-captioning-model',2, _workspace=ws))

image-captioning-tokenizer 2
image-captioning-model 3
image-captioning-tokenizer 1
image-captioning-model 2
image-captioning-model 1
azureml-models/image-captioning-tokenizer/1/tokenizer.pkl
azureml-models/image-captioning-model/2/model.h5


In [ ]:
%pip install azure-cognitiveservices-vision-computervision

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for azure-cognitiveservices-vision-computervision from https://files.pythonhosted.org/packages/f4/cc/d371c24ef8e984c1dfde5c7837c4ab03dc3cdafc321d1f733518e5148567/azure_cognitiveservices_vision_computervision-0.9.0-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [ ]:
%%writefile score.py
import numpy as np
from pickle import load
import os
import json
from azureml.core.model import Model
from PIL import Image
import io
from urllib.request import urlopen
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model as keras_Model
from tensorflow.keras.models import load_model

# Run test image through VGG16 model to get features
def extract_features(photo_bytes):
    base_model = VGG16()
    base_model = keras_Model(inputs=base_model.inputs, outputs=base_model.layers[-2].output)
    photo = Image.open(io.BytesIO(photo_bytes))
    photo = photo.convert('RGB')
    photo = photo.resize((224, 224))
    photo = img_to_array(photo)
    photo = photo.reshape((1, photo.shape[0], photo.shape[1], photo.shape[2]))
    photo = preprocess_input(photo)
    feature = base_model.predict(photo, verbose=0)
    return feature

# Get word in tokenizer using its index
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# Create image caption using custom deep learning model
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo, sequence], verbose=0)
        yhat = np.argmax(yhat)        
        word = word_for_id(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

# Generate image description from Azure Cognitive Services
def cogservices_desc(photo_bytes):
    subscription_key = ''
    endpoint = 'https://.cognitiveservices.azure.com/'
    computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
    image_stream = io.BytesIO(photo_bytes)
    description_result = computervision_client.describe_image_in_stream(image_stream)
    if len(description_result.captions) == 0:
        description_result = "No description detected."
    else:
        description_result = description_result.captions[0].text
    return description_result

# load model assets
def init():
    global model
    global tokenizer
    global max_length
    max_length = 34
    # load model from registered models
    model_path = Model.get_model_path(model_name='image-captioning-model')
    model = load_model(model_path)
    # load tokenizer from registered models
    tokenizer_path = Model.get_model_path(model_name='image-captioning-tokenizer')
    with open(tokenizer_path, 'rb') as handle:
        tokenizer = load(handle)

# define how to handle user data
def run(data):
    try:
        data = json.loads(data)
        captions = dict()
        uri_inputs = data['photo_uri']
        for photo_uri in uri_inputs:
            with urlopen(photo_uri) as content:
                photo_bytes = content.read()
            photo = extract_features(photo_bytes)
            model_description = generate_desc(model, tokenizer, photo, max_length)
            model_description = ' '.join(model_description.split()[1:-1])
            cogserv_description = cogservices_desc(photo_bytes)
            caption_results = {'cogservices_description': cogserv_description, 'model_description': model_description}
            captions[photo_uri] = caption_results
        return json.dumps(captions)
    except Exception as e:
        return json.dumps({"error": str(e)})



Overwriting score.py


In [ ]:
%%writefile score.py
import numpy as np
from pickle import load
import matplotlib.pyplot as plt
from numpy import argmax
import os
import json
from azureml.core.model import Model
from PIL import Image
import io
from urllib.request import urlopen
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.preprocessing.image import load_img
from tensorflow.python.keras.preprocessing.image import img_to_array
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.models import Model as keras_Model
from tensorflow.python.keras.models import load_model

# Run test image through VGG16 model to get features
def extract_features(photo_bytes):
  base_model = VGG16()
  base_model = keras_Model(inputs=base_model.inputs, outputs=base_model.layers[-2].output)
  photo = Image.open(io.BytesIO(photo_bytes))
  photo = photo.convert('RGB')
  photo = photo.resize((224,224))
  photo = img_to_array(photo)
  photo = photo.reshape((1, photo.shape[0], photo.shape[1], photo.shape[2]))
  photo = preprocess_input(photo)
  feature = base_model.predict(photo, verbose=0)
  return feature

# Get word in tokenizer using its index
def word_for_id(integer, tokenizer):
  for word, index in tokenizer.word_index.items():
      if index == integer:
          return word
  return None

# Create image caption using custom deep learning model
def generate_desc(model, tokenizer, photo, max_length):
  in_text = 'startseq'
  for i in range(max_length):
    sequence = tokenizer.texts_to_sequences([in_text])[0]
    sequence = pad_sequences([sequence], maxlen=max_length)
    yhat = model.predict([photo,sequence], verbose=0)
    yhat = argmax(yhat)
    word = word_for_id(yhat, tokenizer)
    if word is None:
        break
    in_text += ' ' + word
    if word == 'endseq':
        break
  return in_text

# Generate image description from Azure Cognitive Services
def cogservices_desc(photo_file_path):
  subscription_key = ''
  endpoint = 'https://.cognitiveservices.azure.com/'
  computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
  description_result = computervision_client.describe_image(photo_file_path)
  if (len(description_result.captions) == 0):
    description_result = "No description detected."
  else:
    description_result = description_result.captions[0].text
  return description_result

# load model assets
def init():
  global model
  global tokenizer
  global max_length
  global conect_str  
  max_length = 34
  # load model from registered models
  model_path = Model.get_model_path(model_name='image-captioning-model')
  model = load_model(model_path)
  # load tokenizer from registered models
  tokenizer_path = Model.get_model_path(model_name='image-captioning-tokenizer')
  tokenizer = load(open(tokenizer_path, 'rb'))

# define how to handle user data
def run(data):
  try:
    data = json.loads(data)
    captions = dict()
    uri_inputs = data['photo_uri']
    for photo_uri in uri_inputs:
      with urlopen(photo_sas_uri) as content:
        photo_bytes = content.read()
      photo = extract_features(photo_bytes)
      model_description = generate_desc(model, tokenizer, photo, max_length)
      model_description = ' '.join(model_description.split()[1:-1])
      cogserv_description = cogservices_desc(photo_uri)
      caption_results = {'cogservices_description': cogserv_description, 'model_description': model_description}
      captions[photo_uri] = caption_results
    return json.dumps(captions)
  except Exception as e:
    error = str(e)
    return error

Overwriting score.py


In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
# Create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("keras")
conda_dep.add_pip_package("azure-cognitiveservices-vision-computervision")
conda_dep.add_pip_package("azure.core")
conda_dep.add_pip_package("keras")
conda_dep.add_pip_package("matplotlib")
conda_dep.add_pip_package("joblib")
conda_dep.add_pip_package("tensorflow")
conda_dep.add_pip_package("azure-storage-blob")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies = conda_dep
# compute target ACI instance
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)
# ACI deployment config
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 8)

model = Model(ws, name='image-captioning-model')
tokenizer = Model(ws, name='image-captioning-tokenizer')
service = Model.deploy(workspace=ws, 
                       name='image-captioning-aci-service6', 
                       models=[model, tokenizer], 
                       inference_config=inference_config, 
                       deployment_config=deployment_config)


/root/.ipykernel/1247/command-2720649002440117-1914697968:30: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [ ]:
print(service.get_logs()) 

In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Ensure that you have logged into Azure and your workspace is set correctly
ws = Workspace(subscription_id='a598dabb-205d-4742-a7fb-3e62549f1960',
               resource_group='image-captioning',
               workspace_name='imagecaptioning-aml')

# Create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("keras")
conda_dep.add_pip_package("azure-cognitiveservices-vision-computervision")
conda_dep.add_pip_package("azure.core")
conda_dep.add_pip_package("matplotlib")
conda_dep.add_pip_package("joblib")
conda_dep.add_pip_package("tensorflow")
conda_dep.add_pip_package("azure-storage-blob")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies = conda_dep

# Inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

# ACI deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=8)

# Reference the registered models
model = Model(ws, name='image-captioning-model')
tokenizer = Model(ws, name='image-captioning-tokenizer')

# Deploy the service
service = Model.deploy(workspace=ws,
                       name='image-captioning-aci-service9',
                       models=[model, tokenizer],
                       inference_config=inference_config,
                       deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)

print(f"Service state: {service.state}")
print(f"Scoring URI: {service.scoring_uri}")


In [ ]:
print(f"Service state: {service.state}")
print(f"Scoring URI: {service.scoring_uri}")


Service state: Unhealthy
Scoring URI: None


In [ ]:
import matplotlib.pyplot as plt
import requests
import json
urls = ['<urls-to-test>']

# Display images
for url in urls:
  response = requests.get(url, stream=True)
  img = Image.open(response.raw)
  plt.imshow(img)
  plt.show()
  
# Send HTTP request
scoring_uri = '<ACI_scoring_uri>'
headers = {'Content-Type':'application/json'}
test_data = json.dumps({'photo_uri': urls}, indent=4)
response = requests.post(scoring_uri, data=test_data, headers=headers)

print(response.status_code)
print(response.elapsed)
print(response.json())


In [ ]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

# extract features from each photo in the directory
def extract_features(directory):
	# load the model
	model = VGG16()
	# re-structure the model
	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	# summarize
	print(model.summary())
	# extract features from each photo
	features = dict()
	for name in listdir(directory):
		# load an image from file
		filename = directory + '/' + name
		image = load_img(filename, target_size=(224, 224))
		# convert the image pixels to a numpy array
		image = img_to_array(image)
		# reshape data for the model
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		# prepare the image for the VGG model
		image = preprocess_input(image)
		# get features
		feature = model.predict(image, verbose=0)
		# get image id
		image_id = name.split('.')[0]
		# store feature
		features[image_id] = feature
		print('>%s' % name)
	return features

# extract features from all images
directory = '/dbfs/mnt/imageforcaptioning/image'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('/dbfs/mnt/imageforcaptioning/features.pkl', 'wb'))

2024-07-01 16:51:41.344060: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-01 16:51:41.346842: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-01 16:51:41.354690: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-01 16:51:41.370339: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-01 16:51:41.370365: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-01 16:51:41.380596: I tensorflow/core/platform/cpu_feature_guard.cc:

2024-07-01 16:51:44.132286: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2024-07-01 16:51:44.177931: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.


    49152/553467096 ━━━━━━━━━━━━━━━━━━━━ 1:06:44 7us/step

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:720)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:439)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:439)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

filename = '/dbfs/mnt/imageforcaptioning/captions.txt'
# load descriptions
doc = load_doc(filename)

In [ ]:
# extract descriptions for images
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# remove filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping

# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 8092 


In [ ]:
import string

def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

In [ ]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))

Vocabulary Size: 8680


In [ ]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

# save descriptions
save_descriptions(descriptions, '/dbfs/mnt/imageforcaptioning/descriptions.txt')

In [ ]:
# import string

# # load doc into memory
# def load_doc(filename):
# 	# open the file as read only
# 	file = open(filename, 'r')
# 	# read all text
# 	text = file.read()
# 	# close the file
# 	file.close()
# 	return text

# # extract descriptions for images
# def load_descriptions(doc):
# 	mapping = dict()
# 	# process lines
# 	for line in doc.split('\n'):
# 		# split line by white space
# 		tokens = line.split()
# 		if len(line) < 2:
# 			continue
# 		# take the first token as the image id, the rest as the description
# 		image_id, image_desc = tokens[0], tokens[1:]
# 		# remove filename from image id
# 		image_id = image_id.split('.')[0]
# 		# convert description tokens back to string
# 		image_desc = ' '.join(image_desc)
# 		# create the list if needed
# 		if image_id not in mapping:
# 			mapping[image_id] = list()
# 		# store description
# 		mapping[image_id].append(image_desc)
# 	return mapping

# def clean_descriptions(descriptions):
# 	# prepare translation table for removing punctuation
# 	table = str.maketrans('', '', string.punctuation)
# 	for key, desc_list in descriptions.items():
# 		for i in range(len(desc_list)):
# 			desc = desc_list[i]
# 			# tokenize
# 			desc = desc.split()
# 			# convert to lower case
# 			desc = [word.lower() for word in desc]
# 			# remove punctuation from each token
# 			desc = [w.translate(table) for w in desc]
# 			# remove hanging 's' and 'a'
# 			desc = [word for word in desc if len(word)>1]
# 			# remove tokens with numbers in them
# 			desc = [word for word in desc if word.isalpha()]
# 			# store as string
# 			desc_list[i] =  ' '.join(desc)

# # convert the loaded descriptions into a vocabulary of words
# def to_vocabulary(descriptions):
# 	# build a list of all description strings
# 	all_desc = set()
# 	for key in descriptions.keys():
# 		[all_desc.update(d.split()) for d in descriptions[key]]
# 	return all_desc

# # save descriptions to file, one per line
# def save_descriptions(descriptions, filename):
# 	lines = list()
# 	for key, desc_list in descriptions.items():
# 		for desc in desc_list:
# 			lines.append(key + ' ' + desc)
# 	data = '\n'.join(lines)
# 	file = open(filename, 'w')
# 	file.write(data)
# 	file.close()

# filename = 'Flickr8k_text/Flickr8k.token.txt'
# # load descriptions
# doc = load_doc(filename)
# # parse descriptions
# descriptions = load_descriptions(doc)
# print('Loaded: %d ' % len(descriptions))
# # clean descriptions
# clean_descriptions(descriptions)
# # summarize vocabulary
# vocabulary = to_vocabulary(descriptions)
# print('Vocabulary Size: %d' % len(vocabulary))
# # save to file
# save_descriptions(descriptions, 'descriptions.txt')

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

In [ ]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

In [ ]:
# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

In [ ]:
from pickle import load

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

# load training dataset (6K)
filename = '/dbfs/mnt/imageforcaptioning/captions.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('/dbfs/mnt/imageforcaptioning/descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('/dbfs/mnt/imageforcaptioning/features.pkl', train)
print('Photos: train=%d' % len(train_features))

Dataset: 8091
Descriptions: train=8091
Photos: train=8091


In [ ]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc
from tensorflow.keras.preprocessing.text import Tokenizer

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 8683


In [ ]:
# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, descriptions, photos, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each image identifier
	for key, desc_list in descriptions.items():
		# walk through each description for the image
		for desc in desc_list:
			# encode the sequence
			seq = tokenizer.texts_to_sequences([desc])[0]
			# split one sequence into multiple X,y pairs
			for i in range(1, len(seq)):
				# split into input and output pair
				in_seq, out_seq = seq[:i], seq[i]
				# pad input sequence
				in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
				# encode output sequence
				out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
				# store
				X1.append(photos[key][0])
				X2.append(in_seq)
				y.append(out_seq)
	return array(X1), array(X2), array(y)

In [ ]:
# calculate the length of the description with the most words
def max_len(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

In [ ]:
# define the captioning model
def define_model(vocab_size, max_length):
	# feature extractor model
	inputs1 = Input(shape=(4096,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(256, activation='relu')(fe1)
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = LSTM(256)(se2)
	# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(256, activation='relu')(decoder1)
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	# summarize model
	print(model.summary())
	plot_model(model, to_file='model.png', show_shapes=True)
	return model

In [ ]:
# define checkpoint callback
from keras.callbacks import ModelCheckpoint
filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.keras'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.layers import Embedding,LSTM,add
# Define the model
def define_model(vocab_size, max_length):
    # Feature extractor model
    inputs1 = Input(shape=(4096,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    
    # Sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    
    # Decoder model
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    
    # Combine the models
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

# Fit the model
model = define_model(8092, 30)
model.fit([X1train, X2train], ytrain, epochs=20, verbose=2, callbacks=[checkpoint], validation_data=([X1test, X2test], ytest))

In [ ]:
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
filename = '/dbfs/mnt/imageforcaptioning/captions.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('/dbfs/mnt/imageforcaptioning/descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('/dbfs/mnt/imageforcaptioning/features.pkl', train)
print('Photos: train=%d' % len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_len(train_descriptions)
print('Description Length: %d' % max_length)
# prepare sequences
X1train, X2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features, vocab_size)
 
# dev dataset
 
# load test set
filename = '/dbfs/mnt/imageforcaptioning/captions.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('/dbfs/mnt/imageforcaptioning/descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('/dbfs/mnt/imageforcaptioning/features.pkl', test)
print('Photos: test=%d' % len(test_features))
# prepare sequences
X1test, X2test, ytest = create_sequences(tokenizer, max_length, test_descriptions, test_features, vocab_size)
 
# fit model
 
# define the model
model = define_model(vocab_size, max_length)
# define checkpoint callback
filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.keras'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# fit model
model.fit([X1train, X2train], ytrain, epochs=20, verbose=2, callbacks=[checkpoint], validation_data=([X1test, X2test], ytest))

java.lang.RuntimeException: abort: DriverClient destroyed
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$4(DriverClient.scala:699)
	at scala.concurrent.Future.$anonfun$flatMap$1(Future.scala:307)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:54)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:77)
	at com.databricks.threading.ContextBoundRunnable.$anonfun$run$2(ContextBoundRunnable.scala:15)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:261)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:257)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext(AttributionContextTracing.scala:46)
	at com.databricks.logging.AttributionContextTracing.withAttributionContext$(AttributionContextTracing.scala:43)
	at com.databricks.threading.ContextBoundRunnable.withAttributionContext(ContextBoundRunnable.scala:6)
	at com.databricks.threading.ContextBoundRunnable.$anonfun$run$1(ContextBoundRunnable.scala:15)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.context.integrity.IntegrityCheckContext$ThreadLocalStorage$.withValue(IntegrityCheckContext.scala:73)
	at com.databricks.threading.ContextBoundRunnable.run(ContextBoundRunnable.scala:14)
	at com.databricks.threading.DatabricksExecutionContext$InstrumentedRunnable.run(DatabricksExecutionContext.scala:36)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:457)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)